In [1]:
cd C:\Users\User\Desktop\sem1_year4\fyp_thing

C:\Users\User\Desktop\sem1_year4\fyp_thing


In [ ]:
# dlib 68 facial landmark keypoint (accurate)

In [ ]:
import tensorflow as tf
import cv2
import os
import matplotlib.pyplot as plt
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
import dlib
from imutils import face_utils
import time

cap = cv2.VideoCapture(0)
count=0
detector = dlib.get_frontal_face_detector()
predictor= dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")
model = tf.keras.models.load_model('new_dataset/model_mobileNet.h5')
status="None"
elapsed=0

while True:
    ret, frame = cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    rects=detector(gray,0)
    
    for rect in rects:
        ## extract mouth region using dlib 68 keypoint
        shape = predictor(gray, rect)
        x1=shape.part(48).x 
        x2=shape.part(54).x 
        y1=shape.part(51).y 
        y2=shape.part(57).y
        cv2.rectangle(frame, (x1-20,y1-20), (x2+20,y2+20), (0,255,0))
        mouth_roi=frame[y1-20:y2+20,x1-20:x2+20]
        final_image = cv2.resize(mouth_roi, (224,224))
        final_image = np.expand_dims(final_image,axis = 0)
        final_image = final_image/255.0
        
        ## prediction using mouth roi
        start=time.time()
        pred_open = model.predict(final_image)
        pred_close = 1-pred_open
        predict = pred_open.round()
        
        if (predict==1):
            status="Open"
            cv2.putText(frame,str(pred_open),(x1-30, y1-30),font, 0.3,(0,255,0),2,cv2.LINE_4)
            end=time.time()
            elapsed=end-start+elapsed

            
        else:
            status="closed"
            cv2.putText(frame,str(pred_close),(x1-30, y1-30),font, 0.3,(0,0,255),2,cv2.LINE_4)
            end=time.time()
            elapsed=0
      
    
    ## feeding decision
    if (elapsed>2):
        font = cv2.FONT_HERSHEY_SIMPLEX
        cv2.putText(frame,'ready to feed!',(100, 10),font, 0.5,(255,0,0),2,cv2.LINE_4)
    else:
        font = cv2.FONT_HERSHEY_SIMPLEX
        cv2.putText(frame,'Please open mouth',(100, 10),font, 0.5,(0,0,255),2,cv2.LINE_4)
    
    elp="{:.2f}".format(elapsed)
    font = cv2.FONT_HERSHEY_SIMPLEX
    cv2.putText(frame,status,(100, 30),font, 0.5,(0,0,0),2,cv2.LINE_4)
    cv2.putText(frame,'Count:'+str(elp),(10, 200),font, 0.5,(0,0,255),2,cv2.LINE_4) 
    cv2.imshow('Open/Closed Mouth Detection', frame)    
    
    if cv2.waitKey(2) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()